# KUIS 2 Pembelajaran Mesin

## Tentang dataset

Pada sebuah lembaga kemanusiaan berbasis teknologi informasi, dibentuk beberapa tim kemanusiaan yang bertugas untuk terjun ke masyarakat memberikan bantuan dalam bentuk aplikasi komputer atau layanan pelatihan IT.  Kegiatn tersebut dilaksanakan setiap tahun dengan ketentuan anggota organisasi tersebut dapat bergabung ke beberapa tim sebagai ketua atau anggota. Terdapat pula anggota yang sudah tidak aktif di organisasi tersebut (terhapus di database mereka —> label N/A).  Organisasi tersebut juga melakukan rekrutmen volunteer, namun tidak semua tim menggunakan bantuan volunteer. Hasil kegiatan dari setiap tim dapat diwujudkan ke dalam sebuah pengakuan hak cipta atau publikasi karya ilmiah atau publikasi berita di media massa.

## Ketentuan
1. Pengerjaan secara individu alias setiap mahasiswa!
2. Terdapat data dengan nilai #N/A, pertimbangkan cara terbaik untuk mengatasi nilai tersebut.
3. Tidak boleh ada duplikasi pekerjaan antar mahasiswa
4. Berikan penjelasan untuk setiap tahapan. Jangan hanya kode!

## Soal
1. Lakukan pemilahan data training dan testing yang sesuai dan proporsional dari data tersebut! (10 poin)
2. Tentukan data validasi! (10 poin)
3. Tentukan secara tepat kolom yang menjadi "label" (20 poin)
4. Terapkan dua jenis metode klasifikasi dan lakukan perbandingan akurasi  dengan wajib melibatkan variabel keterlibatan volunteer sampai variabel luaran-luaran yang dihasilkan! (20 poin)
5. Lakukan tunning hyperparameter berdasarkan hasil klasifikasi pada nomor 3 menggunakan metode Grid Search dan Random Search! Simpulkan parameter apa yang paling baik digunakan berdasarkan metode klasifikasi yang dipilih, dan berapa akurasinya. (40 poin)

In [53]:

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
df = pd.read_excel('data/dataset_quiz.xlsx')

df.head()

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,id_anggota5,keterlibatan_vltr,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


In [54]:
# ubah data nonnumerik menjadi numerik emnggunakan enco
from sklearn import preprocessing

enco = preprocessing.LabelEncoder()

df['keterlibatan_vltr']= enco.fit_transform(df['keterlibatan_vltr'])
df['skema']= enco.fit_transform(df['skema'])
df['luaran_hki']= enco.fit_transform(df['luaran_hki'])
df['jenis_luaran_hki']= enco.fit_transform(df['jenis_luaran_hki'])
df['potensi_hki']= enco.fit_transform(df['potensi_hki'])
df['luaran_lain']= enco.fit_transform(df['luaran_lain'])
df['jenis_luaran_lain']= enco.fit_transform(df['jenis_luaran_lain'])
df['status_luaran_lain']= enco.fit_transform(df['status_luaran_lain'])


### 1. memilahan data training dan testing yang sesuai dan proporsional dari dataset_quiz

In [55]:
# Split data
from sklearn.model_selection import train_test_split

# Split data training dan dan lainnya
# data lainnya, akan kita split lagi menjadi validasi dan testing.
# Rasio yang akan kita gunakan adalah 8:1:1
df_train, df_unseen = train_test_split(df, test_size=0.2, random_state=0)

# Split lagi antara validasi dan testing
df_val, df_test = train_test_split(df_unseen, test_size=0.5, random_state=0)

# Cek masing-masing ukuran data

print(f'Jumlah data asli: {df.shape[0]}')
print(f'Jumlah data train: {df_train.shape[0]}')
print(f'Jumlah data test: {df_test.shape[0]}')

# Cek rasio tiap label
print('=========')
print(f'Jumlah label data asli:\n{df.keterlibatan_vltr.value_counts()}')
print(f'Jumlah label data train:\n{df_train.keterlibatan_vltr.value_counts()}')
print(f'Jumlah label data test:\n{df_test.keterlibatan_vltr.value_counts()}')

Jumlah data asli: 85
Jumlah data train: 68
Jumlah data test: 9
Jumlah label data asli:
0    45
1    40
Name: keterlibatan_vltr, dtype: int64
Jumlah label data train:
0    36
1    32
Name: keterlibatan_vltr, dtype: int64
Jumlah label data test:
0    6
1    3
Name: keterlibatan_vltr, dtype: int64


### 2. menentukan data validasi


In [56]:
print(f'Jumlah data val: {df_val.shape[0]}')
print(f'Jumlah label data val:\n{df_val.keterlibatan_vltr.value_counts()}')

Jumlah data val: 8
Jumlah label data val:
1    5
0    3
Name: keterlibatan_vltr, dtype: int64


### 3. menentukan secara tepat kolom yang menjadi "label"


In [57]:
X = df.iloc[:, 6: 16]
y = df['tahun']

X.head()

,keterlibatan_vltr,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,0,0,2019,1,1,1,2,0,3,1
1,0,0,2019,1,1,1,2,0,3,1
2,0,0,2019,1,1,1,2,0,3,1
3,0,0,2019,1,1,1,2,0,3,1
4,0,0,2019,1,1,1,2,0,3,1


### 4. menerapkan dua jenis metode klasifikasi dan lakukan perbandingan akurasi  dengan wajib melibatkan variabel keterlibatan volunteer sampai variabel luaran-luaran yang dihasilkan


In [58]:
# split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dtc = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dtc.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dtc.predict(X_test)
print(y_pred_dt)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

[2020 2021 2020 2020 2020 2022 2022 2022 2022 2021 2022 2021 2021 2022
 2022 2019 2019]
Test set accuracy: 1.00
Test set accuracy: 1.0


### 5. tunning hyperparameter berdasarkan hasil klasifikasi pada nomor 3 menggunakan metode Grid Search dan Random Search lalu menyimpulkan parameter apa yang paling baik digunakan berdasarkan metode klasifikasi yang dipilih, dan berapa akurasinya.

DecisionTreeClassifier

In [59]:
from sklearn.model_selection import RepeatedKFold

# inisiasi repated k-fold
cv = RepeatedKFold(n_splits=4, n_repeats=3, random_state=42)

In [60]:
# inisiasi model
dt = DecisionTreeClassifier()

# Definisikan hyperparameter yang akan digunakan
# sklearn menerima dalam bentuk dictionary
# nama hyperparamater HARUS SESUAI dengan dokumentasi sklearn
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': list(range(5,11))
}

# inisiasi grid berdasarkan nilai repeated k-fold dan hyperparameter
grid = GridSearchCV(dt, param_grid=params, cv=cv)

# Fit / latih berdasarkan grid
# %timeit merupakan magic command didalam ipython notebook
# yang dapat kita gunakan untuk menghitung waktu komputasi
# cara ini cukup efektif untuk melakukan evaluasi suatu algoritma atau prosedur
%timeit grid.fit(X, y)

# Evaluasi dengan score
score = grid.score(X,y)

print(f'Hasil evaluasi: {score}')
print(f'Konfigurasi hyperparameter: {grid.best_params_}')

509 ms ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Hasil evaluasi: 1.0
Konfigurasi hyperparameter: {'criterion': 'gini', 'max_depth': 5}


In [61]:
# inisiasi model
dtc = DecisionTreeClassifier()

# Definisikan hyperparameter yang akan digunakan
# sklearn menerima dalam bentuk di-ctionary
# nama hyperparamater HARUS SESUAI dengan dokumentasi sklearn
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': list(range(5,11))
}

# inisiasi grid berdasarkan nilai repeated k-fold dan hyperparameter
random = RandomizedSearchCV(dtc, param_distributions=params, cv=cv, n_jobs=-1)

# Fit / latih berdasarkan grid
# %timeit merupakan magic command didalam ipython notebook
# yang dapat kita gunakan untuk menghitung waktu komputasi
# cara ini cukup efektif untuk melakukan evaluasi suatu algoritma atau prosedur
%timeit random.fit(X, y)

# Evaluasi dengan score
score = random.score(X,y)

print(f'Hasil evaluasi: {score}')
print(f'Konfigurasi hyperparameter: {random.best_params_}')

121 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Hasil evaluasi: 1.0
Konfigurasi hyperparameter: {'max_depth': 5, 'criterion': 'entropy'}
